In [1]:
#Importing required libraries
import pandas as pd
import numpy as np
from pandas_datareader import data
import requests
from math import sqrt
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
%matplotlib inline
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sudsawal', api_key='2Y21xZQ1sHCU8JSbrWIp')
import warnings
warnings.filterwarnings('ignore')

In [2]:
portfolio = 1e5 #Portfolio total worth of 1 lakh
tscost = 0.005 #Per trade transaction cost of 0.5%, so if two stocks of 30 each are bought, (1+0.005)*2*30 is charged in total
nstocks = 0

stock_list = ['BHEL.NS','ITC.NS','LUPIN.NS','RELIANCE.NS','IBVENTURES.NS'] #Stock list, to add a stock simply
                                                                           #add the ticker according to Yahoo! Finance
for stock in stock_list:
    nstocks+=1          #Calculates number of stocks according to input stock list

In [3]:
#Function to read Data
def readData(ticker, n):
    stocks.append(data.DataReader(ticker, 'yahoo',start='1/1/2015')) #Taking backtesting period as roughly 5 years
    

In [4]:
#Defines MACD of each ticker
def MACD(df1):
    df1['EMA_small'] = df1['Adj Close'].ewm(span=12).mean()
    df1['EMA_large'] = df1['Adj Close'].ewm(span=26).mean()
    df1['MACD_line'] = df1['EMA_small'] - df1['EMA_large']
    df1['Signal_line'] = df1['MACD_line'].ewm(span=9).mean()
    
    df1['Position'] = 0


In [5]:
#Defines the Position based on a MACD Zero crossover strategy as defined below:

########################################################################################################################
#Entry:  - When MACD line crosses zero line(>0), then long/buy the stock. Similar for squaring off the short position. #
#                                                                                                                      #
#Exit: -   When MACD line crosses zero line(<0), then short/sell the stock. Similar for squaring off the long position.#                           
#                                                                                                                      #
########################################################################################################################


def ZeroCrossover(df1):
    for row in range(len(df1)):
        if(df1['Position'].iloc[row-1]==0):
            if (df1['MACD_line'].iloc[row] < 0):
                df1['Position'].iloc[row] = -1

            if (df1['MACD_line'].iloc[row] > 0):
                df1['Position'].iloc[row] = 1

            
        elif (df1['Position'].iloc[row-1]==1):
            if (df1['MACD_line'].iloc[row] < 0):
                df1['Position'].iloc[row] = 0
            if (df1['MACD_line'].iloc[row] > 0):
                df1['Position'].iloc[row] = 1

        else:
            if (df1['MACD_line'].iloc[row] > 0):
                df1['Position'].iloc[row] = 0
            if (df1['MACD_line'].iloc[row] < 0):
                df1['Position'].iloc[row] = -1

    

In [6]:
#Defines the Position based on a MACD Signal Crossover strategy as defined below:

########################################################################################################################
#Entry: -When MACD line crosses signal line and greater than it, then long/buy the stock.                              #
#       - Similar for squaring off the short position.                                                                 #
#Exit:  -When MACD line crosses signal line and less than it, then short/sell the stock.                               #
#       -Similar for squaring off the long position.                                                                   #                           
#                                                                                                                      #
########################################################################################################################


def SignalCrossover(df1):
    for row in range(len(df1)):
        if(df1['Position'].iloc[row-1]==0):
            if (df1['MACD_line'].iloc[row] < df1['Signal_line'].iloc[row]):
                df1['Position'].iloc[row] = -1

            if (df1['MACD_line'].iloc[row] > df1['Signal_line'].iloc[row]):
                df1['Position'].iloc[row] = 1

            
        elif (df1['Position'].iloc[row-1]==1):
            if (df1['MACD_line'].iloc[row] < df1['Signal_line'].iloc[row]):
                df1['Position'].iloc[row] = 0
            if (df1['MACD_line'].iloc[row] > df1['Signal_line'].iloc[row]):
                df1['Position'].iloc[row] = 1

        else:
            if (df1['MACD_line'].iloc[row] > df1['Signal_line'].iloc[row]):
                df1['Position'].iloc[row] = 0
            if (df1['MACD_line'].iloc[row] < df1['Signal_line'].iloc[row]):
                df1['Position'].iloc[row] = -1

    

In [7]:
#Defines the Position based on a MACD Divergence(in a given window) strategy as defined below:

########################################################################################################################
#Entry: -When closing price records a higher high, but the MACD records a lower high. then long/buy the stock.         #
#       -Similar for squaring off the short position.                                                                  #
#Exit:  -When closing price records a lower low, but the MACD records a higher low. Then short/sell the stock.         #
#       -Similar for squaring off the long position.                                                                   #                           
#                                                                                                                      #
########################################################################################################################

def Divergence(df1):
    for row in range(1,len(df1)):
        if(df1['Position'].iloc[row-1]==0):
            if (df1['Adj Close'].rolling(20).max()[row] > df1['Adj Close'].rolling(20).max()[row-1]) and (df1['MACD_line'].rolling(20).max()[row] <= df1['MACD_line'].rolling(20).max()[row-1]):
                df1['Position'].iloc[row] = 1

            if (df1['Adj Close'].rolling(20).min()[row] < df1['Adj Close'].rolling(20).min()[row-1]) and (df1['MACD_line'].rolling(20).min()[row] >= df1['MACD_line'].rolling(20).min()[row-1]):
                df1['Position'].iloc[row] = -1

            
        elif (df1['Position'].iloc[row-1]==1):
            if (df1['Adj Close'].rolling(20).max()[row] > df1['Adj Close'].rolling(20).max()[row-1]) and (df1['MACD_line'].rolling(20).max()[row] <= df1['MACD_line'].rolling(20).max()[row-1]):
                df1['Position'].iloc[row] = 1
            if (df1['Adj Close'].rolling(20).min()[row] < df1['Adj Close'].rolling(20).min()[row-1]) and (df1['MACD_line'].rolling(20).min()[row] >= df1['MACD_line'].rolling(20).min()[row-1]):
                df1['Position'].iloc[row] = 0

        else:
            if (df1['Adj Close'].rolling(20).min()[row] < df1['Adj Close'].rolling(20).min()[row-1]) and (df1['MACD_line'].rolling(20).min()[row] >= df1['MACD_line'].rolling(20).min()[row-1]):
                df1['Position'].iloc[row] = -1
            if (df1['Adj Close'].rolling(20).max()[row] > df1['Adj Close'].rolling(20).max()[row-1]) and (df1['MACD_line'].rolling(20).max()[row] <= df1['MACD_line'].rolling(20).max()[row-1]):
                df1['Position'].iloc[row] = 0

    

In [8]:
def computer():
  s = (len(stocks[0]),nstocks) #setting the dimensions of the weights vector depending on the available data points
  weights = np.zeros(s) 
  sumwts = 0

  for row in range(len(stocks[0])):
      i = 0
      for i in range(nstocks):
          sumwts+=np.abs(stocks[i]['Position'].iloc[row]) #checking how many positions were taken on that day
      for i in range(nstocks):
          if sumwts>0:
              #weights assigned as:
              #weights = (portfolio*position)/(no.of positions * adj close on that day * (1+transac cost))
              weights[row][i]=(portfolio*stocks[i]['Position'].iloc[row])/((sumwts*stocks[i]['Adj Close'].iloc[row])*(1+tscost))
          else:
              weights[row][i]=0
  i = 0


  for i in range(nstocks):
      stocks[i]['Market Return'] = (stocks[i]['Adj Close'] - stocks[i]['Adj Close'].shift(1)) #Absolute market return

    
  #Calculating portfolio return
  stocks[0]['Strategy Return']=0
  i = 0
  for i in range(nstocks):
      stocks[0]['Strategy Return']+=stocks[i]['Market Return'] * weights[:,i] #multiplying market return of each
                                                                              #ticker with its weight

# **Zero Crossover**

In [9]:
#Iterating over all the tickers to read data, generate MACD and zero crossover conditions and generate positions based on strategy
i = 0
stocks = []
for ticker in stock_list:
    readData(ticker,i)
    MACD(stocks[i])
    ZeroCrossover(stocks[i])
    i+=1
computer()

IndexError: single positional indexer is out-of-bounds

In [ ]:
trace = go.Scatter(
  text = ((100 * (stocks[0]['Strategy Return'][64:].cumsum())/portfolio).values),
  x = stocks[0]['Strategy Return'][64:].cumsum().index,
  y = (stocks[0]['Strategy Return'][64:].cumsum())
)
data1 = [trace]
layout = go.Layout(
    title='Hover over the points to see the text'
)
fig1 = go.Figure(data=data1, layout=layout)
py.iplot(data1, filename='basic-line')

# **Signal Line Crossover**

In [10]:
#Iterating over all the tickers to read data, generate MACD and give the positions based on strategy
i = 0
stocks = []
for ticker in stock_list:
    readData(ticker,i)
    MACD(stocks[i])
    SignalCrossover(stocks[i])
    i+=1
computer()

ValueError: operands could not be broadcast together with shapes (1107,) (1106,) 

In [ ]:
trace = go.Scatter(
  text = ((100 * (stocks[0]['Strategy Return'][64:].cumsum())/portfolio).values),
  x = stocks[0]['Strategy Return'][64:].cumsum().index,
  y = (stocks[0]['Strategy Return'][64:].cumsum())
)
data2 = [trace]
layout = go.Layout(
    title='Hover over the points to see the text'
)
fig2 = go.Figure(data=data2, layout=layout)
py.iplot(data2, filename='basic-line2')

# **MACD Divergence**

In [ ]:
#Iterating over all the tickers to read data, generate MACD and generate positions based on Divergence strategy
i = 0
stocks = []
for ticker in stock_list:
    readData(ticker,i)
    MACD(stocks[i])
    Divergence(stocks[i])
    i+=1
computer()

In [ ]:
trace = go.Scatter(
  text = ((100 * (stocks[0]['Strategy Return'][64:].cumsum())/portfolio).values),
  x = stocks[0]['Strategy Return'][64:].cumsum().index,
  y = (stocks[0]['Strategy Return'][64:].cumsum())
)
data3 = [trace]
layout = go.Layout(
    title='Hover over the points to see the text'
)
fig3 = go.Figure(data=data3, layout=layout)
py.iplot(data3, filename='basic-line3')